In [1]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_pickle("../../data/processed/final_dataset.pkl")
df

,amount,delayed_flag,past_total_settlements,past_delay_rate,avg_settlement_time,past_avg_amount,amount_vs_user_avg,avg_delay_last_5
17,658.34,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
12,199.43,0,1,0.000000,2.000000,658.340000,0.302929,2.000000
20,266.89,1,2,0.000000,2.000000,428.885000,0.622288,2.000000
7,329.44,0,3,0.333333,4.666667,374.886667,0.878772,4.666667
4,1091.29,1,4,0.250000,3.750000,363.525000,3.001967,3.750000
...,...,...,...,...,...,...,...,...
8073,447.35,0,29,0.137931,2.034483,449.305172,0.995648,1.200000
8074,400.28,0,30,0.133333,2.033333,449.240000,0.891016,1.200000
8065,485.93,0,31,0.129032,1.967742,447.660645,1.085487,1.200000
8059,152.47,0,32,0.125000,1.906250,448.856562,0.339685,0.800000


In [2]:
X = df.drop(['delayed_flag'], axis='columns')
y = df['delayed_flag']

In [3]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf

StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

auc_scores = []
for train_index, test_index in skf.split(X, y):
    
    X_train_fold = X.iloc[train_index]
    X_valid_fold = X.iloc[test_index]

    y_train_fold = y.iloc[train_index]
    y_valid_fold = y.iloc[test_index]

    # print(X_train_fold.shape)
    # print(X_valid_fold.shape)
    # print(y_train_fold.shape)
    # print(y_valid_fold.shape)
    
    scaler = StandardScaler()
    scaler.fit(X_train_fold)
    X_train_scaled = scaler.transform(X_train_fold)
    X_valid_scaled = scaler.transform(X_valid_fold)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train_scaled, y_train_fold)
    probab = model.predict_proba(X_valid_scaled)[:,1]

    roc = roc_auc_score(y_valid_fold, probab)
    auc_scores.append(roc)
    print(roc)

0.7001445103045572
0.6735364559235772
0.7112738027197925
0.6973391757160095
0.6697990648071448


In [5]:
import numpy as np
mean_auc= (sum(auc_scores)/len(auc_scores))
mean_auc 
std_auc = np.std(auc_scores)

In [7]:
mean_auc
std_auc

0.6904186018942162